In [1]:
% cd /mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [2]:
import sys
sys.path.append('script/training/')

from train import EntityDatasetPhraseFeat

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import pandas as pd
import numpy as np

In [33]:
class RegionEntityDatasetGT(EntityDatasetPhraseFeat):

    def __init__(self, data_file,
                 phrase_feature_file,
                 unique_phrase_file,
                 img_data_file,
                 san_check=False,
                 preload=False,
                 skip=None):
        
        super(RegionEntityDatasetGT, self).__init__(data_file, phrase_feature_file, unique_phrase_file, san_check, preload=preload)
        img_feat_f, img_meta_f = img_data_file
        
        self.im_feat = np.load(img_feat_f)
        self.roi_meta = pd.read_csv(img_meta_f)
    
    def get_region_feat(self, i):
        img = self._image_id[i]
        roi1, roi2 = self._roi_label[i]
        idx1 = self.roi_meta.query("image == %i & roi == '%s'" % (img, roi1))['Unnamed: 0']
        idx2 = self.roi_meta.query("image == %i & roi == '%s'" % (img, roi2))['Unnamed: 0']

        f1 = self.im_feat[idx1]
        f2 = self.im_feat[idx2]
        
        return f1, f2
        
    def get_example(self, i):
        # get region features
        r1, r2 = self.get_region_feat(i)
        
        # get phrase feature
        x1, x2 = self._get_entity(i)

        # get label
        y = self._get_label(i)
        y = np.asarray(y, dtype=np.int32)
        
        image = np.vstack([r1, r2])
        gt_r1 = 0
        gt_r2 = 1
        
        return image, x1, x2, gt_r1, gt_r2, y

In [34]:
data = RegionEntityDatasetGT('data/gt-roi/phrase_pair_val.csv',
        'data/phrase_feat/wea/val.npy',
       'data/phrase_misc/val_uniquePhrases',
       ['data/region_feat/gt-roi-frcnn/val.npy', 'data/region_feat/gt-roi-frcnn/meta_val.csv'])

data/gt-roi/phrase_pair_val.csv


In [35]:
item = data.get_example(199)

In [30]:
img = data._image_id[0]
roi1, roi2 = data._roi_label[0]
idx1 = data.roi_meta.query("image == %i & roi == '%s'" % (img, roi1))['Unnamed: 0']
idx2 = data.roi_meta.query("image == %i & roi == '%s'" % (img, roi2))['Unnamed: 0']

f1 = data.im_feat[idx1]
f2 = data.im_feat[idx2]

In [32]:
f1.shape

(1, 4096)

In [13]:
pd.read_csv('data/gt-roi/phrase_pair_val.csv')[:3]

,Unnamed: 0,Unnamed: 0.1,image,cap_id1,cap_id2,phr_id1,phr_id2,category1,category2,phrase1,phrase2,org_phrase1,org_phrase2,ytrue,roi1,roi2
0,0,0,100652400,0,1,197,198,people,clothing,man,hard+hat,A man,a hard hat,False,"(45, 53, 203, 110)","(44, 59, 66, 88)"
1,1,1,100652400,0,1,197,199,people,clothing,man,caution+vest,A man,a caution vest,False,"(45, 53, 203, 110)","(69, 62, 119, 98)"
2,2,2,100652400,0,1,197,200,people,scene,man,street,A man,the street,False,"(45, 53, 203, 110)","(90, 1, 500, 374)"


In [27]:
loc = data.roi_meta.query("image == %i & roi == '%s'" % (img, roi1))['Unnamed: 0']

In [28]:
loc

0    0
Name: Unnamed: 0, dtype: int64

In [29]:
np.arange(9)[loc]

array([0])